In [1]:
USER_FLAG = '--user'
!pip install -U pip
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.3.0 --upgrade
!pip3 install {USER_FLAG} kfp --upgrade
!pip install google_cloud_pipeline_components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.2/105.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [9]:
PROJECT_ID = 'churn-smu'
REGION = 'asia-southeast1'
BUCKET_NAME = 'practice-smu-123'

! gcloud config set project {PROJECT_ID}
! gcloud services enable  compute.googleapis.com \
                          containerregistry.googleapis.com \
                          aiplatform.googleapis.com \
                          cloudbuild.googleapis.com \
                          cloudfunctions.googleapis.com \
                          dataflow.googleapis.com

Updated property [core/project].
Operation "operations/acat.p2-352395168404-aa8602bf-4a6f-43da-a84e-d47e095e3e46" finished successfully.


In [10]:
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import Artifact, Dataset, Input, Model, Output, Metrics, ClassificationMetrics, component, OutputPath, InputPath
import kfp
from kfp.v2 import compiler
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [11]:
PATH=%env PATH
%env PATH = {PATH}:/home/jupyter/.local/bin

PIPELINE_ROOT = "gs://" + f'{BUCKET_NAME}/churn/pipeline_root_churn'
DATA_ROOT_TRAIN = "gs://" + f"{BUCKET_NAME}/churn/data/dev"
DATA_ROOT_EVAL = "gs://" + f"{BUCKET_NAME}/churn/data/val"
DATA_ROOT_SERVE = "gs://" + f"{BUCKET_NAME}/churn/data/serve"

print(f"Project ID: {PROJECT_ID}")
print(f"Pipeline Root: {PIPELINE_ROOT}")
print(f"GCS Bucket Name: {BUCKET_NAME}")
print(f"DataRoot Train Directory: {DATA_ROOT_TRAIN}")

env: PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/home/jupyter/.local/bin
Project ID: churn-smu
Pipeline Root: gs://practice-smu-123/churn/pipeline_root_churn
GCS Bucket Name: practice-smu-123
DataRoot Train Directory: gs://practice-smu-123/churn/data/dev


In [12]:
! python -m pip install --user virtualenv
! echo "create env"
! python -m venv vertex_venv
! echo "Add kernel to jupyter"
! ipython kernel install --name "vertex_env" --user 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.2/461.2 kB 36.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
create env
Error: Command '['/content/drive/MyDrive/vertex_venv/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
Add kernel to jupyter
Installed kernelspec vertex_env in /root/.local/share/jupyter/kernels/vertex_env


In [24]:
ingest = kfp.components.load_component_from_file("./yaml files/data_ingest_component.yaml")
tfdv_validate = kfp.components.load_component_from_file("./yaml files/py/tfdv_component_py.yaml")
impute_and_store = kfp.components.load_component_from_file("./yaml files/data_impute_component.yaml")
impute = kfp.components.load_component_from_file("./yaml files/py/impute_component_py.yaml")
enc_and_scl = kfp.components.load_component_from_file("./yaml files/py/encode_and_scale_component_py.yaml")
enc_and_scl_store = kfp.components.load_component_from_file("./yaml files/data_enc_scl_store_component.yaml")
hyperparameter_tuning = kfp.components.load_component_from_file("./yaml files/hyperparameter_tuning_component.yaml")
train_churn = kfp.components.load_component_from_file("./yaml files/py/train_component_py.yaml")
check_model_metrics = kfp.components.load_component_from_file("./yaml files/py/check_model_metrics_component_py.yaml")
deploy_churn_model = kfp.components.load_component_from_file("./yaml files/py/model_churn_prediction_component_py.yaml")
test_endpoint = kfp.components.load_component_from_file("./yaml files/py/test_endpoint_py.yaml")

In [ ]:
@component(base_image = "python:3.9")

def check_model_metrics(model: Input[Model],
                        thresholds_dict_str: str,
                        ) -> NamedTuple("output",[("deploy",str)]):

    import json

    def threshold_check(val1,val2):
        cond = "False"
        if val1 > val2:
            cond = "True"
        return cond

    metric = model.metadata["f1_score"]
    thresholds_dict = json.loads(thresholds_dict_str)
    deploy = threshold_check(float(metric), thresholds_dict['f1_score'])

    return (deploy,)

In [ ]:
@component(packages_to_install=["google-cloud-aiplatform", "sklearn", "kfp"],
           base_image = "python:3.9",
           output_component_file = "model_churn_prediction_component.yaml")

def deploy_churn_model(model: Input[Model],
                       project: str, region: str,
                       serving_container_image_uri: str,
                   #    vertex_endpoint: Output[Artifact],
                       vertex_model: Output[Model]) -> NamedTuple("output",[("endpoint",str)]):
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    DISPLAY_NAME = "churn_prediction_v2"
    MODEL_NAME = "churn_rf_v2"
    ENDPOINT_NAME = "churn_endpoint_v2"

    def create_endpoint():
        endpoints = aiplatform.Endpoint.list(
            filter = 'display_name = "{}"'.format(ENDPOINT_NAME),
            order_by = 'create_time desc',
            project = project,
            location = region,
        )

        if len(endpoints) > 0:
            endpoint = endpoints[0]
        else:
            endpoint = aiplatform.Endpoint.create(display_name = ENDPOINT_NAME,
                                                  project = project,
                                                  location = region)
            
        return endpoint
    
    endpoint = create_endpoint()
    endpoint_info = endpoint.resource_name.split('/')[-1]
    
    model_upload = aiplatform.Model.upload(display_name = DISPLAY_NAME,
                                           artifact_uri = model.uri.replace("model",""),
                                           serving_container_image_uri = serving_container_image_uri,
                                           serving_container_health_route=f'/v1/models/{MODEL_NAME}',
                                           serving_container_predict_route=f'/v1/models/{MODEL_NAME}:predict',
                                           serving_container_environment_variables = {"MODEL_NAME":MODEL_NAME,},
                                           )
    
    model_deploy = model_upload.deploy(machine_type = "n1-standard-4",
                                      endpoint = endpoint,
                                      traffic_split = {"0":100},
                                      deployed_model_display_name = DISPLAY_NAME)
    
    vertex_model.uri = model_deploy.resource_name
    return (endpoint_info,)

In [15]:
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = "pipeline-churn-prediction-job{}".format(TIMESTAMP)

In [ ]:
@component(packages_to_install = ["google-cloud-aiplatform"],
           base_image = "python:3.9") 

def test_endpoint(project: str,
                  location: str,
                  endpoint: str,) -> NamedTuple("output",[("endpoint_status",str)]):
    
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=location)

    instance = [[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,
                 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,
                 0.0, 0.0, -0.210, -0.695, 0.341, -0.528, 1.282, -1.632, 1.568, 1.309]]

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instance)
    assert int(prediction[0][0]) in [0, 1], "Prediction Error"
    return ("OK",)

## Pipeline for development (Generation of Artifacts)

In [ ]:
@dsl.pipeline(pipeline_root = PIPELINE_ROOT, name = "pipeline-churn-prediction")

def pipeline(data_root_train: str = DATA_ROOT_TRAIN + '/dev.csv',
             data_root_dev: str = DATA_ROOT_EVAL + '/eval.csv',
             bucket_name: str = BUCKET_NAME,
             project: str = PROJECT_ID,
             region: str = REGION,
             display_name: str = DISPLAY_NAME,
             api_endpoint: str = REGION + '-aiplatform.googleapis.com',
             threshold_dict_str: str = '{"f1_score":0.6}',
             serving_container_image_uri: str = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
             gcs_temp_location: str = f'gs://{BUCKET_NAME}/churn/tmp',
             gcs_staging_location: str = f'gs://{BUCKET_NAME}/churn/staging',
             ):
    
    data_train_op = ingest(data_root_train)

    data_validation_op = tfdv_validate(data_train_op.outputs['dataset'],
                                       project_id = project, region = region, bucket = bucket_name,
                                       gcs_temp_location = gcs_temp_location,
                                       gcs_staging_location = gcs_staging_location)
    
    impute_train_op = impute_and_store(data_validation_op.outputs['output_data'], 
                                         bucket = bucket_name)
    
    encode_train_op = enc_and_scl_store(impute_train_op.outputs['post_impute_dataset'],
                                            bucket_name = bucket_name)

    hypertune_op = hyperparameter_tuning(encode_train_op.outputs['post_enc_dataset'],
                                        bucket_name = bucket_name)

    train_model_op = train_churn(encode_train_op.outputs['post_enc_dataset'],
                                bucket_name = bucket_name).after(hypertune_op)

    check_model_metrics_op = check_model_metrics(model = train_model_op.outputs['model'],
                                                thresholds_dict_str = threshold_dict_str)
    
    with dsl.Condition(
        check_model_metrics_op.outputs['deploy'] == "True",
        name = "deploy-churn",):
        deploy_model_op = deploy_churn_model(model = train_model_op.outputs['model'],
                                            project = project,
                                            region = region,
                                            serving_container_image_uri = serving_container_image_uri)
        test_endpoint_op = test_endpoint(project = project,
                                         location = region,
                                         endpoint = deploy_model_op.outputs['endpoint'])
        
compiler.Compiler().compile(pipeline_func = pipeline,
                            package_path = 'churn_prediction.json')
start_pipeline = pipeline_jobs.PipelineJob(display_name = 'churn-prediction-pipeline',
                                           template_path = 'churn_prediction.json',
                                           enable_caching = False,
                                           location = REGION,
                                           project = PROJECT_ID,
                                           )
start_pipeline.run()

## Pipeline for retraining

In [ ]:
@dsl.pipeline(pipeline_root = PIPELINE_ROOT, name = "pipeline-churn-prediction")

def pipeline(data_root_train: str = DATA_ROOT_TRAIN + '/dev.csv',
             data_root_dev: str = DATA_ROOT_EVAL + '/eval.csv',
             bucket_name: str = BUCKET_NAME,
             project: str = PROJECT_ID,
             region: str = REGION,
             display_name: str = DISPLAY_NAME,
             api_endpoint: str = REGION + '-aiplatform.googleapis.com',
             threshold_dict_str: str = '{"f1_score":0.6}',
             serving_container_image_uri: str = "europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
             gcs_temp_location: str = f'gs://{BUCKET_NAME}/churn/tmp',
             gcs_staging_location: str = f'gs://{BUCKET_NAME}/churn/staging',
             ):
    
    data_train_op = ingest(data_root_train)

    data_validation_op = tfdv_validate(data_train_op.outputs['dataset'],
                                       project_id = project, region = region, bucket = bucket_name,
                                       gcs_temp_location = gcs_temp_location,
                                       gcs_staging_location = gcs_staging_location)
    
    impute_train_op = impute(data_validation_op.outputs['output_data'], 
                                         bucket = bucket_name)
    
    encode_train_op = enc_and_scl(impute_train_op.outputs['post_impute_dataset'],
                                            bucket_name = bucket_name)

    train_model_op = train_churn(encode_train_op.outputs['post_enc_dataset'],
                                bucket_name = bucket_name)

    check_model_metrics_op = check_model_metrics(model = train_model_op.outputs['model'],
                                                 thresholds_dict_str = threshold_dict_str)
    
    with dsl.Condition(
        check_model_metrics_op.outputs['deploy'] == "True",
        name = "deploy-churn",):
        deploy_model_op = deploy_churn_model(model = train_model_op.outputs['model'],
                                            project = project,
                                            region = region,
                                            serving_container_image_uri = serving_container_image_uri)
        test_endpoint_op = test_endpoint(project = project,
                                         location = region,
                                         endpoint = deploy_model_op.outputs['endpoint'])
        
compiler.Compiler().compile(pipeline_func = pipeline,
                            package_path = 'churn_prediction.json')
start_pipeline = pipeline_jobs.PipelineJob(display_name = 'churn-prediction-pipeline',
                                           template_path = 'churn_prediction.json',
                                           enable_caching = False,
                                           location = REGION,
                                           project = PROJECT_ID,
                                           )
start_pipeline.run()

## Batch Prediction Job

In [ ]:
job_display_name = "churn-batch-prediction-job"
MODEL_NAME="churn-prediction"
BUCKET_URI="gs://practice-smu/churn/data/serve"
input_file_name="serve4.csv"

MODEL_ID=!(gcloud ai models list --region=$REGION --filter=display_name=$MODEL_NAME)
MODEL_ID=MODEL_ID[2].split(" ")[0]

model_resource_name = f'projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}'
gcs_source= f"{BUCKET_URI}/{input_file_name}"
gcs_destination_prefix=f"{BUCKET_URI}/output"
print(f'Model Resource Name: {model_resource_name}')
print(f'GCS Source: {gcs_source}')
print(f'GCS Destination: {gcs_destination_prefix}')

Model Resource Name: projects/practice-smu/locations/asia-southeast1/models/3217223799430709248
GCS Source: gs://practice-smu/churn/data/serve/serve4.csv
GCS Destination: gs://practice-smu/churn/data/serve/output


In [ ]:
def batch_prediction_job(
    project: str,
    location: str,
    model_resource_name: str,
    job_display_name: str,
    gcs_source: str,
    gcs_destination_prefix: str,
    machine_type: str,
    starting_replica_count: int = 1, # The number of nodes for this batch prediction job. 
    max_replica_count: int = 1,    
):   
    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_resource_name)
    batch_prediction_job = model.batch_predict(
        job_display_name=job_display_name,
        instances_format='csv', #json

        gcs_source=[f"{BUCKET_URI}/{input_file_name}"],
        gcs_destination_prefix=f"{BUCKET_URI}/output",
        machine_type=machine_type, # must be present      
    )
    batch_prediction_job.wait()
    print(batch_prediction_job.display_name)
    print(batch_prediction_job.state)
    return batch_prediction_job

batch_prediction_job(PROJECT_ID, REGION, model_resource_name, job_display_name, gcs_source, gcs_destination_prefix, machine_type="n1-standard-4")

In [ ]:
from google.cloud import aiplatform

def endpoint_predict_sample(
    project: str,
    location: str,
    instances: list,
    endpoint: str,
):
    aiplatform.init(project=project, location=location)

    instance = [[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,
                 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,
                 0.0, 0.0, -0.210, -0.695, 0.341, -0.528, 1.282, -1.632, 1.568, 1.309]]

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    return prediction

In [ ]:
x = endpoint_predict_sample(project = PROJECT_ID,
                        location = REGION,
                        instances = [example.tolist()],
                        endpoint = '4598069666429009920')

## Cloud Scheduler

In [ ]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(
                project_id=PROJECT_ID,
                region=REGION,
                )

SERVICE_ACCOUNT = (
    "352395168404@developer.gserviceaccount.com" # Replace the Xs with your generated service-account.
)
response = api_client.create_schedule_from_job_spec(
    enable_caching=True,
    job_spec_path="churn_prediction.json",
    schedule="0 0 4 * *",
    time_zone="Asia/Singapore",  # change this as necessary
    parameter_values={"display_name": 'test1'},
    pipeline_root=PIPELINE_ROOT,  
    service_account=SERVICE_ACCOUNT,    
)